In [ ]:
import sys
sys.path.append("../source")

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()

from Match_Analytics import Match
from Tracking_Visualization import draw_pitch, plot_sliding_window, play_match
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle
import os
working_dir = os.getcwd()

In [ ]:
match_1 = Match(data_source = 'metrica-sports', match_id = 1, name = "Sample_Game_1", field_dimen=(106., 68.))
match_2 = Match(data_source = 'metrica-sports', match_id = 2, name = "Sample_Game_1", field_dimen=(106., 68.))
matches = [match_1, match_2]

In [ ]:
def Fold_X(row):
    if row[-1]<0:
        return row.multiply(-1)
    
    return row


def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir


def Compute_GS_Array(x, y):
    positions = []
    
    for i in range(len(x)):
        positions.append([x[i], y[i]])
        
    return np.array(positions)


def bic_scorer(fitted_gmm, X):
    return -fitted_gmm.bic(X) #el retorno com a - perquè pel GridSearch el més gran és el millor

    
def isNaN(num):
    return num != num


def Check_NaN(ball):
    if isNaN(ball) == True:
        return 1
    else:
        return 0


def plot_gmm(labels, colors, X, n_components, label=True, ax=None):
    source = ColumnDataSource(dict(x = X[:,0], y = X[:,1], color=colors[labels], label=labels))
    pitch = draw_pitch(pattern="stripes_horizontal_vertical", stripes_number = 5, grass_alpha=0.6)
    pitch.circle(x="x", y="y", color="color", legend_group="label", source=source, size=6)
    pitch.legend.location="right"
    pitch.add_layout(pitch.legend[0], "right")
    show(pitch)
    return pitch

    
def GaussianMixture_f(x, y, n_components, n_init, covariance_type, max_iter):
    t0 = time.time()
    positions = Compute_GS_Array(x,y)
    gmm = GaussianMixture(n_components=n_components, n_init=n_init, covariance_type=covariance_type, max_iter=max_iter)
    fit = gmm.fit(positions)
    labels = fit.predict(positions)
    means = gmm.means_
    covs = gmm.covariances_
    print(means)
    print(labels)
    colors = plt.cm.hsv(np.linspace(0.1,0.9,n_components))
    print(f"Time Elapsed: {time.time()-t0} seconds")
    plot_gmm(labels, colors, positions, n_components)
    return labels, means, positions, covs


def Frame_Modification(series, count):
    nanspos = []
    for idx, val in series.items():
        if val == 1:
            nanspos.append(count)
        else:
            nanspos.append(0)
            
    return nanspos

In [ ]:
def Detect_NaN_Corners(filt, xframe, yframe):      
    xtrack = []
    ytrack = []
    labs = []
    for i in range(len(xframe)):
        #if no NaN is found, just keep the actual DataFrames
        if filt[i] == 0:
            xframe[i].reset_index(inplace=True)
            yframe[i].reset_index(inplace=True)
            xtrack.append(xframe[i])
            ytrack.append(yframe[i])
            labs.append([])
            
        #otherwise: concatenate (X,Y) positions
        #get NaN row positions from columns ball_x/y
        #keep those positions, remove them and return them
        else:
            fin_concat = pd.concat([xframe[i], yframe[i]], axis=1)
            fin_concat.reset_index(inplace=True)
            xind = fin_concat["ball_x"].index.values[fin_concat["ball_x"].apply(np.isnan)].tolist()
            yind = fin_concat["ball_y"].index.values[fin_concat["ball_y"].apply(np.isnan)].tolist()
            indexes = list(np.unique(xind+yind))
            labs.append(indexes)
            xtrack.append(fin_concat.filter(like="_x"))
            ytrack.append(fin_concat.filter(like="_y"))
    #labs is returned as if an initial/final position is deprecated due to the presence of ball NaNs
    #those corner positions should be discarded from final/initial position DataFrames
    return xtrack, ytrack, labs


def Modify_Frames(nanX, nanY, x, y, count, pos, position, tracks, filt, i):
    if filt[i] != 0:
        xnans = Frame_Modification(nanX, count)
        ynans = Frame_Modification(nanY, count)
        nans = np.array(xnans)+np.array(ynans)
        nans = np.where(nans != 0, count, 0)
        position[i] = pos[i]+nans
        x[i] = tracks[i].loc[position[i]].filter(like="_x")
        y[i] = tracks[i].loc[position[i]].filter(like="_y")
    
    return x[i], y[i], position[i]


def Avoid_NaNs(corner_reference, corners_filtered, ini_pos, fin_pos, fin_pos_ball, trackings):
    #GET TRACKINGS PER SEPARATE OF (X,Y) COORDINATES FOR INITIAL AND FINAL POSITIONS
    xini = [trackings[i].loc[ini_pos[i]].filter(like="_x") for i in range(len(trackings))]
    yini = [trackings[i].loc[ini_pos[i]].filter(like="_y") for i in range(len(trackings))]
    xref = [trackings[i].loc[corner_reference[i]].filter(like="_x") for i in range(len(trackings))]
    yref = [trackings[i].loc[corner_reference[i]].filter(like="_y") for i in range(len(trackings))]
    xfin = [trackings[i].loc[fin_pos[i]].filter(like="_x") for i in range(len(trackings))]
    yfin = [trackings[i].loc[fin_pos[i]].filter(like="_y") for i in range(len(trackings))]

    #AS ALL CORNERS WILL BE MOVED TO THE SAME ONE, WE ARE ONLY INTERESTED IN BALL FINAL POSITIONS
    xballfin = [trackings[i].loc[fin_pos_ball[i]].filter(like="_x") for i in range(len(trackings))]
    yballfin = [trackings[i].loc[fin_pos_ball[i]].filter(like="_y") for i in range(len(trackings))]
    
    #COPIES OF THE STARTING AND ENDING FRAMES TO SEARCH FOR NOT NAN VALUES
    final_pos = np.copy(fin_pos)
    initial_pos = np.copy(ini_pos)
    corn_ref = np.copy(corner_reference)
    final_pos_ball = np.copy(fin_pos_ball)
    count = 0
    counter = -5
    
    print("Inspecting if NaN present in ball position")
    
    while True:
        if count == 0: print("Searching if not NaN is present in -5+fin_pos --> fin_pos+5")

        filt1 = np.zeros((len(xini)))
        filt2 = np.zeros((len(xini)))
        filt3 = np.zeros((len(xini)))
        thresh1 = 0
        thresh2 = 0
        thresh3 = 0
        
        for i in range(len(xini)):
            #RETURN 1 IF A NAN IS FOUND IN BALL POSITION
            checkx = xref[i]["ball_x"].apply(Check_NaN)
            checky = yref[i]["ball_y"].apply(Check_NaN)
            checkfinx = xfin[i]["ball_x"].apply(Check_NaN)
            checkfiny = yfin[i]["ball_y"].apply(Check_NaN)
            checkballx = xballfin[i]["ball_x"].apply(Check_NaN)
            checkbally = yballfin[i]["ball_y"].apply(Check_NaN)
            
            filt1[i] = checkx.sum()+checky.sum()
            filt2[i] = checkfinx.sum()+checkfiny.sum()
            filt3[i] = checkballx.sum()+checkbally.sum()
            
            #SEARCH IN -5+THE_FRAME ---> THE_FRAME+5 FOR NOT NAN VALUES
            xref[i], yref[i], corn_ref[i] = Modify_Frames(checkx,checky,xref,yref,counter,corner_reference,
                                                          corn_ref,trackings,filt1,i)
            xfin[i], yfin[i], final_pos[i] = Modify_Frames(checkfinx, checkfiny, xfin, yfin, counter,
                                                           fin_pos, final_pos, trackings, filt2, i)
            xballfin[i], yballfin[i], final_pos_ball[i] = Modify_Frames(checkballx, checkbally, xballfin, yballfin,
                                                        counter, fin_pos_ball, final_pos_ball, trackings, filt3, i)
            thresh1 += filt1[i]
            thresh2 += filt2[i]
            thresh3 += filt3[i]

        counter += 1
        count += 1
        print(count)
        if ((thresh1 == 0) & (thresh2 == 0) & (thresh3 == 0)):
            print(f"New frame found with no NaNs at + ({-5+count})")
            
            for i in range(len(xini)):
                xini[i].reset_index(inplace=True)
                xini[i] = xini[i].filter(like="_x")
                yini[i].reset_index(inplace=True)
                yini[i] = yini[i].filter(like="_y")
                xfin[i].reset_index(inplace=True)
                xfin[i] = xfin[i].filter(like="_x")
                yfin[i].reset_index(inplace=True)
                yfin[i] = yfin[i].filter(like="_y")
                xballfin[i].reset_index(inplace=True)
                xballfin[i] = xballfin[i].filter(like="_x")
                yballfin[i].reset_index(inplace=True)
                yballfin[i] = yballfin[i].filter(like="_y")
                xref[i].reset_index(inplace=True)
                xref[i] = xref[i].filter(like="_x")
                yref[i].reset_index(inplace=True)
                yref[i] = yref[i].filter(like="_y")
            
            
            return xini, yini, xfin, yfin, xballfin, yballfin, xref, yref
        
        elif count == 9:
            print("Any alternative found: deleting corner NaN data")
            #GET LABELS OF EACH CORNERS MATCH THAT CONTAINS NANS
            xini, yini, nouse = Detect_NaN_Corners(filt1, xini, yini) 
            xref, yref, lab1 = Detect_NaN_Corners(filt1, xref, yref) 
            xfin, yfin, lab2 = Detect_NaN_Corners(filt2, xfin, yfin)
            xballfin, yballfin, lab3 = Detect_NaN_Corners(filt3, xballfin, yballfin)

            for i in range(len(xini)):
                #CONCATENATE RESPECTIVE (X,Y) POSITIONS TO REMOVE ROWS GIVEN BY THE LABELS
                concat0 = pd.concat([xref[i], yref[i]], axis=1)
                concat1 = pd.concat([xini[i], yini[i]], axis=1)
                concat2 = pd.concat([xfin[i], yfin[i]], axis=1)
                concat3 = pd.concat([xballfin[i], yballfin[i]], axis=1)
                
                #GET UNIQUE LABELS IN CASE SOMEONE IS REPEATED. IF NOT, DROP WOULD RAISE AN ERROR AS IT WOULD NOT
                #FIND THE LABEL: IT HAS ALREADY BEEN REMOVED
                delete_indexes = list(np.unique(lab1[i]+lab2[i]+lab3[i]))
                concat0.drop(delete_indexes, axis=0, inplace=True)
                concat1.drop(delete_indexes, axis=0, inplace=True)
                concat2.drop(delete_indexes, axis=0, inplace=True)
                concat3.drop(delete_indexes, axis=0, inplace=True)

                #SEPARATE AGAIN DATAFRAMES
                xref[i] = concat0.filter(like="_x")
                yref[i] = concat0.filter(like="_y")
                xini[i] = concat1.filter(like="_x")
                yini[i] = concat1.filter(like="_y")
                xfin[i] = concat2.filter(like="_x")
                yfin[i] = concat2.filter(like="_y")
                xballfin[i] = concat3.filter(like="_x")
                yballfin[i] = concat3.filter(like="_y")
                
            return xini, yini, xfin, yfin, xballfin, yballfin, xref, yref
            


def Get_Conversion(element):
    if element<0:
        return -1
    else:
        return 1
    
            
def Fold_X_or_Y(xdfini, ydfini, ydffin, ballyfin, refx, refy):
    
    for i in range(len(ydfini)):
        #COMPUTE COLUMN OF 1 OR -1 DEPENDING ON BALL Y INITIAL POSITION
        refy[i]["Conversion"] = refy[i]["ball_y"].apply(Get_Conversion).astype(float)
        refx[i]["Conversion"] = refx[i]["ball_x"].apply(Get_Conversion).astype(float)

#         ydfini[i]["Conversion"] = ydfini[i]["ball_y"].apply(Get_Conversion).astype(float)

        for column in ydffin[i]:
            #MULTIPLY EACH ROW ELEMENT BY THE CONVERSION FACTOR
                    #if conversion = -1 ---> the corner is kicked from the negative pitch site and positions will
                                                #be reversed
                    #otherwise
            ydfini[i][column] = ydfini[i][column]*refy[i]["Conversion"]
            ydffin[i][column] = ydffin[i][column]*refy[i]["Conversion"]
            ballyfin[i][column] = ballyfin[i][column]*refy[i]["Conversion"]
            
        for column in xdfini[i]:
            xdfini[i][column] = xdfini[i][column]*refx[i]["Conversion"]

    return xdfini, ydfini, ydffin, ballyfin


def Get_Player_Samples(xini,yini,xfin,yfin,initial_ids,initial_gauss,fin_ids,final_gauss):
    
    df = pd.DataFrame(zip(xini,yini,initial_ids,initial_gauss,xfin,yfin,fin_ids,final_gauss), columns=["xi","yi","# corner_i","ini_gauss","xf","yf","# corner_f","fin_gauss"])
    df_vals = pd.DataFrame(zip(initial_gauss, final_gauss), columns=["ini_gauss", "fin_gauss"])
    vals = np.unique(df_vals)
    n_corners = df["# corner_i"].max()+1
    samples = []
    for i in range(n_corners):
        df2 = df[(df["# corner_i"] == i) & (df["# corner_f"] == i)]
        df2 = df2.loc[:,["ini_gauss", "fin_gauss"]]
        df2["val"] = 1
        df3 = pd.DataFrame(0, index=vals, columns=vals)
        f = df3.index.get_indexer
        df3.values[f(df2.ini_gauss), f(df2.fin_gauss)] = df2.val.values
        samples.append(df3.replace(0,-1))
    
    return samples


def Get_Ball_Samples(bxfin,byfin,final_gauss):
    ini_id = np.zeros((len(bxfin)), dtype="int")
    df = pd.DataFrame(zip(bxfin,byfin,ini_id,final_gauss,np.linspace(0,len(bxfin),len(bxfin)+1,dtype="int")), columns=["xf","yf","ini_gauss","fin_gauss","# corner"])
    df_vals = pd.DataFrame(zip(ini_id, final_gauss), columns=["ini", "fin"])
    vals = np.unique(df_vals)
    samples = []
    
    for i in range(len(df)):
        df2 = df[(df["# corner"] == i)]
        df2 = df2.loc[:,["ini_gauss", "fin_gauss"]]
        df2["val"] = 1
        df3 = pd.DataFrame(0, index=[0], columns=vals)
        f = df3.index.get_indexer
        df3.values[f(df2.ini_gauss), f(df2.fin_gauss)] = df2.val.values
        samples.append(df3.replace(0,-1))
        
    return samples



def Detect_Short_Corners(ydf, yfilter):      

    labs = []
    for j in range(len(ydf)):

        if ydf["ball_y"][j] > yfilter:
            labs.append(j)

    return labs


def Delete_Short_Corners(xi, yi, xf, yf, bx, by, yfilter):
    
    for i in range(len(by)):
        
        xi[i] = xi[i].reset_index(drop=True)
        yi[i] = yi[i].reset_index(drop=True)
        xf[i] = xf[i].reset_index(drop=True)
        yf[i] = yf[i].reset_index(drop=True)
        bx[i] = bx[i].reset_index(drop=True)
        by[i] = by[i].reset_index(drop=True)
        
        labs = Detect_Short_Corners(by[i], yfilter)
        print("short corner detected if neq 0", len(labs))
        if len(labs) == 0:
            continue
        else:
            xi[i].drop(labs, axis=0, inplace=True)
            yi[i].drop(labs, axis=0, inplace=True)
            xf[i].drop(labs, axis=0, inplace=True)
            yf[i].drop(labs, axis=0, inplace=True)
            bx[i].drop(labs, axis=0, inplace=True)
            by[i].drop(labs, axis=0, inplace=True)
        
    return xi, yi, xf, yf, bx, by

In [ ]:
################################################
    #matches: list of matches
    #pj_start_frame: frame to determine initial position
    #pj_end_frame: frame to determine final position of the players
    #ball_end_frame: frame to determine final position of the ball
    #xfilter: x position to discard players that do not interfere in the corner kick
            #xfilter \in [0,53]
                    #0: center of the field
                    #53: goal position
    
    #in case NaNs are found in ball positions, 10 frames around the desired one would be inspected.
    #if any NaN is still present, it would discard the corner
################################################
                    
def Filter_Corners(matches, team, corner_reference, pj_start_frame, pj_end_frame, ball_end_frame, xfilter, yfilter):
    
    trackings = [match.tracking_home for match in matches] #get list of preprocessed matches
    corners_filtered = [match.events[(match.events["Subtype"]=="CORNER KICK") & (match.events["Team"]==team)] for match in matches] #get all corner events of all matches
    initial_pos = [np.array(corner_pos["Start Frame"].values)+pj_start_frame for corner_pos in corners_filtered] #initial positions of all corners
    corner_ref = [np.array(corner_pos["Start Frame"].values)+corner_reference for corner_pos in corners_filtered]
    final_pos = [np.array(corner_pos["Start Frame"].values)+pj_end_frame for corner_pos in corners_filtered] #final positions of all corners
    final_pos_ball = [np.array(corner_pos["Start Frame"].values)+ball_end_frame for corner_pos in corners_filtered] #final positions of balls
    
    #process data avoidind NaNs in ball position as a NaN would suppose no relevant information
    ini_tracking_x, ini_tracking_y, fin_tracking_x, fin_tracking_y, fin_tracking_ballx, fin_tracking_bally, refx, refy = Avoid_NaNs(corner_ref, corners_filtered, initial_pos, final_pos, final_pos_ball, trackings)

    #fold positions to move them to the same hal
    fin_tracking_x = [track.apply(Fold_X, axis=1) for track in fin_tracking_x]
    fin_tracking_ballx = [track.apply(Fold_X, axis=1) for track in fin_tracking_ballx]

    #fold also y positions to move all corners to the same one depending on the initial Y position of the ball
    ini_tracking_x, ini_tracking_y, fin_tracking_y, fin_tracking_bally = Fold_X_or_Y(ini_tracking_x, ini_tracking_y, fin_tracking_y, fin_tracking_bally, refx, refy)
    ini_tracking_x, ini_tracking_y, fin_tracking_x, fin_tracking_y, fin_tracking_ballx, fin_tracking_bally = Delete_Short_Corners(ini_tracking_x, ini_tracking_y, fin_tracking_x, fin_tracking_y, fin_tracking_ballx, fin_tracking_bally, yfilter)
    
    #once NaNs filtered, concatenate dfs to obtain all corners in one df
    ini_tracking_x = pd.concat([df for df in ini_tracking_x]).reset_index(drop=True)
    ini_tracking_y = pd.concat([df for df in ini_tracking_y]).reset_index(drop=True)
    fin_tracking_x = pd.concat([df for df in fin_tracking_x]).reset_index(drop=True)
    fin_tracking_y = pd.concat([df for df in fin_tracking_y]).reset_index(drop=True)
    fin_tracking_ballx = pd.concat([df for df in fin_tracking_ballx]).reset_index(drop=True)
    fin_tracking_bally = pd.concat([df for df in fin_tracking_bally]).reset_index(drop=True)
    print(len(ini_tracking_x))
    #get list of lists of all positions separately and the IDs of the corners
    xpos_ini = [row[i] for idx, row in ini_tracking_x.iterrows() for i in range(len(row)) if i!=(len(row)-1)]
    ypos_ini = [row[i] for idx, row in ini_tracking_y.iterrows() for i in range(len(row)) if i!=(len(row)-1)]
    ini_ids = [i for i, (idx, row) in enumerate(ini_tracking_x.iterrows()) for j in range(len(row)) if j!=(len(row)-1)]
    xpos_fin = [row[i] for idx, row in fin_tracking_x.iterrows() for i in range(len(row)) if i!=(len(row)-1)]
    ypos_fin = [row[i] for idx, row in fin_tracking_y.iterrows() for i in range(len(row)) if i!=(len(row)-1)]
    fin_ids = [i for i, (idx, row) in enumerate(fin_tracking_x.iterrows()) for j in range(len(row)) if j!=(len(row)-1)]
    ballx_fin = [row[i] for idx, row in fin_tracking_ballx.iterrows() for i in range(len(row)) if i==(len(row)-1)]
    bally_fin = [row[i] for idx, row in fin_tracking_bally.iterrows() for i in range(len(row)) if i==(len(row)-1)]    

    #filter all positions depending on XFILTER, that is, only players that interfere in the corner are kept
    #the filter is used depending on the initial x position
    xini_filter = [xpos_ini[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    yini_filter = [ypos_ini[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    xfin_filter = [xpos_fin[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    yfin_filter = [ypos_fin[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    ini_ids_filter = [ini_ids[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    fin_ids_filter = [fin_ids[i] for i in range(len(xpos_ini)) if ((xpos_ini[i] > xfilter) & (ypos_ini[i] < yfilter))]
    
    #if some data is lost during the tracking (NaNs), remove those positions
    df = pd.DataFrame(zip(xini_filter, yini_filter), columns=["x_ini", "y_ini"])
    df.dropna(inplace=True)
    xini = df["x_ini"].tolist()
    yini = df["y_ini"].tolist()
    df = pd.DataFrame(zip(xfin_filter, yfin_filter), columns=["x_fin", "y_fin"])
    df.dropna(inplace=True)
    xfin = df["x_fin"].tolist()
    yfin = df["y_fin"].tolist()

    return xini, yini, xfin, yfin, ballx_fin, bally_fin, ini_ids_filter, fin_ids_filter

In [ ]:
def Sampling_Corners(matches, team_as_string, corner_reference, pj_ini_frame, pj_fin_frames, ini_components, fin_components, xfilter, yfilter):
    
    for fin_frame in pj_fin_frames:
        for comp in ini_components:
            for comp2 in fin_components:
                print(f"Comb: {fin_frame}_{comp}_{comp2}")
                file1 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"pos_ini.npy", "wb")
                file2 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"pos_fin.npy", "wb")
                file3 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"pos_ball.npy", "wb")
                gauss1 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"gauss_ini.npy", "wb")
                gauss2 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"gauss_fin.npy", "wb")
                gauss3 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"gauss_ball.npy", "wb")
                means1 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"mean_ini.npy", "wb")
                means2 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"mean_fin.npy", "wb")
                means3 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"mean_ball.npy", "wb")
                covs1 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"cov_ini.npy", "wb")
                covs2 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"cov_fin.npy", "wb")
                covs3 = open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+"cov_ball.npy", "wb")
                x,y,xf,yf,bx,by,iniid,finid = Filter_Corners(matches, team_as_string, corner_reference, pj_ini_frame, fin_frame, fin_frame, xfilter, yfilter)
                lab, mean, pos, cov = GaussianMixture_f(x,y,comp,20,"full",200)
                lab2, mean2, pos2, cov2 = GaussianMixture_f(xf,yf,comp2,20,"full",200)
                lab3, mean3, pos3, cov3 = GaussianMixture_f(bx,by,comp2,20,"full",200)
                print(len(pos), len(pos2), len(pos3))
                ball_samples = Get_Ball_Samples(bx,by,lab3)
                samples = Get_Player_Samples(x,y,xf,yf,iniid,lab,finid,lab2)
                np.save(file1, pos)
                np.save(file2, pos2)
                np.save(file3, pos3)
                np.save(gauss1, lab)
                np.save(gauss2, lab2)
                np.save(gauss3, lab3)
                np.save(means1, mean)
                np.save(means2, mean2)
                np.save(means3, mean3)
                np.save(covs1, cov)
                np.save(covs2, cov2)
                np.save(covs3, cov3)
                
                cont = 0
                for i in range(len(samples)):
                    cont+=1
                    concatenated = pd.concat([samples[i], ball_samples[i]])
                    pickle.dump(concatenated, open(create_dir(working_dir+f"/sampled/pj{fin_frame}_ball{fin_frame}_{comp}_{comp2}_{comp2}/")+f"corner_{cont}.bin", "wb"))
                    
    file1.close()
    file2.close()
    file3.close()
    gauss1.close()
    gauss2.close()
    gauss3.close()
    means1.close()
    means2.close()
    means3.close()
    covs1.close()
    covs2.close()
    covs3.close()

In [ ]:
G_ini = [6,8]
G_fin_and_ball = [6,8]
PJ_and_ball_fin = [25,37]

Sampling_Corners(matches,"Home",0,-12,PJ_and_ball_fin,G_ini,G_fin_and_ball,29,25)

In [ ]:
matrix_example = pickle.load(open(working_dir+"/sampled/pj25_ball25_6_6_6/corner_9.bin", "rb"))
matrix_example